In [ ]:
TODO: сейчас мердж не сделан! Ты итерируешься по всем данным
Хотя должен только по тем, где х1 == нан, но по всему графу

TODO: код походу неправильно робит, считает по классическому фит предикт, хотя должен был нормально обучиться

In [35]:
import numpy as np
import pandas as pd

In [54]:
train_df = pd.read_csv("data/short_train.csv")
test_df = pd.read_csv("data/short_test.csv")

In [55]:
# train_df['t'].hist(bins = 500)

In [56]:
train_df["t"] = train_df["t"].fillna(-1)
test_df["t"] = test_df["t"].fillna(-1) 

In [57]:
train_df['target'] = train_df['x1']

In [58]:
train_df.columns

Index(['ego_id', 'u', 'v', 't', 'x1', 'x2', 'x3', 'target'], dtype='object')

### Сымитируем реальную выборку, где нужно предсказывать в среднем 20% связей для графа. В каждом графе сделаем x1 целевой метрикой, но предиктить будем только этих 20% связей. Сделаем различные фичи для этих связей.

### Почему предиктим только 20%, а не все 100%? -Потому что в этом случае многие фичи будут содержать даталик. Соот-но, если мы будем вначале "выкидывать"(заменять на nan) x1, а потом только строить дополнительные фичи, то даталика не будет, а также бонусом получается имитация test, где фичи строятся с nan значениями в x1.


In [59]:
all_nan_indexes = np.array([])
for (graph_index, sub_df) in train_df.groupby('ego_id'):
    rows_amount = sub_df.shape[0]
    first_row_index = sub_df.index[0]
    
    nan_indexes = np.random.choice(rows_amount,int(0.2*rows_amount))+first_row_index 
    all_nan_indexes = np.concatenate((all_nan_indexes,nan_indexes))
    train_df.loc[nan_indexes,'target'] = np.nan


In [60]:
train_dataset = train_df.iloc[all_nan_indexes].drop(columns=['target'])
test_dataset = test_df


In [61]:
train_dataset

,ego_id,u,v,t,x1,x2,x3
249,0,240,235,21.1,1.427472,0.000000,0.0
71,0,239,151,45.8,0.002231,0.000000,0.0
715,0,133,100,81.9,1.491979,0.000000,0.0
1262,0,20,18,372.1,0.000000,0.000000,0.0
1025,0,224,191,84.1,1.678250,1.098612,0.0
...,...,...,...,...,...,...,...
1999419,25769803916,228,20,18.9,0.000000,0.000000,0.0
1996839,25769803916,98,224,3.7,0.673966,0.000000,0.0
1997852,25769803916,47,37,30.8,0.009444,0.000000,0.0
1999601,25769803916,77,287,17.7,0.208448,0.000000,0.0


### Объединим данные из attr.csv с нашим датасетом для того, чтобы можно было сделать больше различных признаков

In [43]:
attr = pd.read_csv('data/attr.csv')

In [44]:
import tqdm
ans = {}
attr['v'] = attr['u']

#df_ = df.copy()
#df['age'] = 0
for col in attr.columns[2:]:
    train_dataset[col] = 0
    train_dataset['u_'+col] = 0
    train_dataset['v_'+col] = 0
    
    #     ans[col] = []
#     df['u_'+ col] = 0    
#     df['v_'+ col] = 0    

u_age = []
amount = 0
am1 = 0
temp = 0
for ego_id in tqdm.tqdm(train_dataset['ego_id'].unique()):
    temp = train_dataset.loc[train_dataset['ego_id'] ==ego_id,:]
    attr_temp = attr.loc[attr['ego_id']==ego_id,:]

    for col in attr.columns[2:]:
      if col !='v':  
        temp1 =  temp.merge(attr_temp.loc[:,['u', col]], on = 'u', how = 'left',copy=False)
        train_dataset.loc[train_dataset['ego_id']==ego_id,'u_'+col] = temp1[col+"_y"].to_numpy()#'age_y']
        
        
        temp1 =  temp.merge(attr_temp.loc[:,['v', col]], on = 'v', how = 'left',copy=False)
        train_dataset.loc[train_dataset['ego_id']==ego_id,'v_'+col] = temp1[col+"_y"].to_numpy()#'age_y']


100%|██████████| 962/962 [00:49<00:00, 19.56it/s]


In [45]:
#drop остаточные колонки
train_dataset = train_dataset.drop(columns = ['age',
        'city_id',  'sex',  'school',  'university', 'u_v', 'v_v'])

In [47]:
#То же самое для test_dataset:
import tqdm
ans = {}
attr['v'] = attr['u']

#df_ = df.copy()
#df['age'] = 0
for col in attr.columns[2:]:
    test_dataset[col] = 0
    test_dataset['u_'+col] = 0
    test_dataset['v_'+col] = 0
    
    #     ans[col] = []
#     df['u_'+ col] = 0    
#     df['v_'+ col] = 0    

u_age = []
amount = 0
am1 = 0
temp = 0
for ego_id in tqdm.tqdm(test_dataset['ego_id'].unique()):
    temp = test_dataset.loc[test_dataset['ego_id'] ==ego_id,:]
    attr_temp = attr.loc[attr['ego_id']==ego_id,:]

    for col in attr.columns[2:]:
      if col !='v':  
        temp1 =  temp.merge(attr_temp.loc[:,['u', col]], on = 'u', how = 'left',copy=False)
        test_dataset.loc[test_dataset['ego_id']==ego_id,'u_'+col] = temp1[col+"_y"].to_numpy()#'age_y']
        
        
        temp1 =  temp.merge(attr_temp.loc[:,['v', col]], on = 'v', how = 'left',copy=False)
        test_dataset.loc[test_dataset['ego_id']==ego_id,'v_'+col] = temp1[col+"_y"].to_numpy()#'age_y']
#drop остаточные колонки
test_dataset = test_dataset.drop(columns = ['age',
        'city_id',  'sex',  'school',  'university', 'u_v', 'v_v'])

100%|██████████| 506/506 [00:34<00:00, 14.58it/s]


In [48]:
test_dataset

,ego_id,u,v,t,x1,x2,x3,u_age,v_age,u_city_id,v_city_id,u_sex,v_sex,u_school,v_school,u_university,v_university
0,8,20,0,185.7,3.839089e-04,0.0,0.0,37.0,36,979281502.0,979281502,2.0,2,783569353.0,213987831,372487423.0,845825535
1,8,131,0,161.4,4.034464e-01,0.0,0.0,35.0,36,-1.0,979281502,2.0,2,-1.0,213987831,-1.0,845825535
2,8,73,0,127.0,8.554643e-05,0.0,0.0,58.0,36,-1.0,979281502,2.0,2,-1.0,213987831,-1.0,845825535
3,8,0,0,594.5,2.886418e-01,0.0,0.0,36.0,36,979281502.0,979281502,2.0,2,213987831.0,213987831,845825535.0,845825535
4,8,63,0,127.0,4.281692e-07,0.0,0.0,39.0,36,979281502.0,979281502,2.0,2,301269505.0,213987831,845825535.0,845825535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,42949673174,51,0,-1.0,0.000000e+00,0.0,1.0,51.0,66,613782284.0,555859534,1.0,1,-1.0,674336154,-1.0,-1
999996,42949673174,61,0,5.3,2.490737e+00,0.0,0.0,66.0,66,930240784.0,555859534,1.0,1,30680726.0,674336154,216656971.0,-1
999997,42949673174,62,0,-1.0,2.349250e+00,0.0,0.0,66.0,66,255703164.0,555859534,1.0,1,-1.0,674336154,-1.0,-1
999998,42949673174,31,0,8.2,NaN,0.0,0.0,18.0,66,528441128.0,555859534,1.0,1,91357846.0,674336154,275866820.0,-1


#### Разделим добавляемые фичи на локальные, глобальные и смешанные. Локальные признаки - признаки, полученные по информации с информации о ребре(строчке из датасета). Глобальные - информация о графе в целом. Смешанные - совмещает как информацию о ребре, так и информацию о графе.

In [ ]:
#Признак: "одинаковая школа". Если люди учатся в одинаковой школе, то тем для общения(взаимодействия)
# больше
#train_df['same_school'] =  

362364

### t>15 - is old friends. ЗДЕСЬ ОПИСАНИЕ


In [22]:
train_dataset['isOldFriends'] = (train_dataset['t']>15).astype(np.int8)
test_dataset['isOldFriends'] = (test_dataset['t']>15).astype(np.int8)


### Нормализуем данные

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

y = train_dataset['x1']
X = train_dataset.drop(columns=['x1'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, shuffle=False, random_state=42)#shuffle = False, т.к граф представляется последовательно
sc.fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)



## После подготовки данных, подберем модель и обучим ее. 

### Данную задачу удалось свести к задаче классического  ML, поэтому в качестве моделей будут использоваться XGBoost и LightGBM, т.к эти модели хорошо показывают себя в решении задач классического ML, где присутствует большое разнообразие фич.

In [50]:
from sklearn.metrics import mean_squared_error

In [51]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train, y_train)
mean_squared_error(y_val, model.predict(X_val), squared=False)

0.8049031904980167

In [52]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)
mean_squared_error(y_val, model.predict(X_val), squared=False)


0.7926710255958029

### Submission

In [13]:
test_df_nan_x1 = test_df[test_df['x1'].isna()]
test_df_nan_x1_np = sc.transform( test_df_nan_x1.drop(columns = ['x1']))
test_df_nan_x1.loc[:,'x1'] = model.predict(test_df_nan_x1_np)

C:\Users\boris\AppData\Local\Temp\ipykernel_19008\3880865614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_nan_x1.loc[:,'x1'] = model.predict(test_df_nan_x1_np)
